The program is referenced and modified from:
> https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  

Reference article explaining how to improve the program:
> https://david-exiga.medium.com/music-generation-using-lstm-neural-networks-44f6780a4c5  

Additional Chinese program explanation:
> https://github.com/xitu/gold-miner/blob/master/TODO1/how-to-generate-music-using-a-lstm-neural-network-in-keras.md

In [1]:
# Install dependency 
# music21 Introduction: https://juejin.cn/post/7063827463058489352
! pip install music21 keras tensorflow[and-cuda] 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 87.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 20.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 91.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.2/577.2 MB 19.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 MB 39.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [3]:
# For reading files
import glob
# array processing
import numpy
from matplotlib import pyplot
# keras for building deep learning model
import keras
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# 使用music21來進行midi檔案的操作
from music21 import converter, instrument, note, chord, stream, duration
# music21 介紹: https://juejin.cn/post/7063827463058489352


### 從目錄下的 midi 文件中獲取所有的音符和和弦

In [4]:
"""
Extract all notes and chords from MIDI files in a directory
"""

import glob
from music21 import converter, instrument, note, chord

# Initialize lists to store notes, rests, and other data
notes = []
rests = []
notes_without_rests = []
rest_to_duration = {}

# Use glob to read MIDI files from the specified path:
# "./"               - Current directory
# "midi_songs/"      - Folder named midi_songs
# "*.mid"            - All files with a .mid extension
# Notes are the musical pitches; this script reads notes from MIDI files into a list

for file in glob.glob("./midi_songs/*.mid"):  # Read all MIDI files from the folder
    # Parse the MIDI file using music21
    midi = converter.parse(file)

    print(f"Parsing {file}")

    notes_to_parse = None

    try:  # If the MIDI has instrument parts, select the first one
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:  # If no instrument parts, get notes directly
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        # If the element is a Note, get its pitch
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
            notes_without_rests.append(str(element.pitch))
        # If the element is a Rest, get its name and duration
        elif isinstance(element, note.Rest):
            notes.append(str(element.fullName))
            rests.append(str(element.fullName))
            rest_to_duration[element.fullName] = element.duration.quarterLength
        # If the element is a Chord, get its pitches as integers for easier processing
        elif isinstance(element, chord.Chord):
            pitches = '.'.join(str(n) for n in element.normalOrder)
            notes.append(pitches)
            notes_without_rests.append(pitches)


Parsing ./midi_songs/Bohemian Rhapsody.mid
Parsing ./midi_songs/Never-Gonna-Give-You-Up.mid
Parsing ./midi_songs/Tokyo Ghoul - Unravel.mid
Parsing ./midi_songs/One Night In Tokyo.mid
Parsing ./midi_songs/Linkin Park - One Step Closer.mid
Parsing ./midi_songs/The-Final-Countdown.mid
Parsing ./midi_songs/Daft Punk - Aerodynamic.mid


### 準備神經網絡使用的輸入輸出

In [5]:
"""
Prepare input and output for the neural network
"""

import numpy
from tensorflow.keras.utils import to_categorical

# Get the number of unique note names (including rests)
n_vocab = len(set(notes))
# Sorted list of unique note names (including rests)
pitch_and_rest = sorted(set(item for item in notes))
# Sorted list of unique note names (excluding rests)
pitchnames = sorted(set(item for item in notes_without_rests))
# Sorted list of unique rest names
restnames = sorted(set(item for item in rests))
# Create a dictionary mapping each note/rest to an integer for training
note_to_int = {note: number for number, note in enumerate(pitch_and_rest)}

"""
# Alternative: Map notes and rests separately
note_to_int = {note: number for number, note in enumerate(pitchnames)}
note_to_int.update({rest: number + len(pitchnames) for number, rest in enumerate(restnames)})
"""

print("\n===== Explanation of Variables =====\n")
print("notes: A list storing all notes in the sheet music as strings.")
print(f"Total number of notes (including rests): {len(notes)}")
print(f"Total number of notes (excluding rests): {len(pitchnames)}")
print(f"Total number of unique note/rest names: {n_vocab}")
print(f"Total number of unique rest names: {len(set(rests))}")
print(f"Unique note names: {pitchnames}")
print(f"Mapping of note names to integers: {note_to_int}")

# Length of the input sequence for training (number of notes per sequence)
sequence_length = 100

# Initialize input and output sequences for training
network_input = []
network_output = []

# ===== Create input sequences and corresponding outputs from the notes =====
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]  # Input sequence of notes
    sequence_out = notes[i + sequence_length]  # The next note as output
    
    # Convert notes in the input sequence to their corresponding integer values
    network_input.append([note_to_int[char] for char in sequence_in])
    # Convert the output note to its corresponding integer value
    network_output.append(note_to_int[sequence_out])

print("\n===================\n")
print(f"Total notes in 'notes': {len(notes)}")
print(f"Each {sequence_length} notes form one training sample.")
print(f"Total training samples (network_input): {len(network_input)}, each with {len(network_input[0])} integers.")
print(f"Total output samples (network_output): {len(network_output)}, each representing the next note's integer.")
print("\n===================\n")
print("Notes from position sequence_length-10 to sequence_length:", notes[sequence_length-10:sequence_length])
print("Their corresponding integers:", [note_to_int[char] for char in notes[sequence_length-10:sequence_length]])
print("")
print(f"Last 10 integers of network_input[0]: {network_input[0][sequence_length-10:sequence_length]}")
print(f"Last 10 integers of network_input[1]: {network_input[1][sequence_length-10:sequence_length]}")
print(f"Last 10 integers of network_input[2]: {network_input[2][sequence_length-10:sequence_length]}")
print("First three integers in network_output:", network_output[0:3])

# Total number of training patterns
n_patterns = len(network_input)

# ===== Reshape input to be compatible with LSTM layers =====
normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# Normalize input values
normalized_input = normalized_input / float(n_vocab)

# Convert output to a one-hot encoded format suitable for categorical_crossentropy
network_output = to_categorical(network_output, n_vocab)

print("\n===== Reshaped Data =====\n")
print("Shape of normalized_input:", normalized_input.shape)
print("Shape of network_output:", network_output.shape)



===== Explanation of Variables =====

notes: A list storing all notes in the sheet music as strings.
Total number of notes (including rests): 1568
Total number of notes (excluding rests): 51
Total number of unique note/rest names: 75
Total number of unique rest names: 24
Unique note names: ['0', '0.5', '2.7', '3.7', '3.9', '5.10', '5.8', '7.0', '8.0', '9.0', 'A2', 'A4', 'A5', 'A6', 'B-4', 'B-5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C#2', 'C#4', 'C#5', 'C1', 'C3', 'C4', 'C5', 'C6', 'D2', 'D5', 'D6', 'E-5', 'E-6', 'E2', 'E4', 'E5', 'E6', 'F#2', 'F#4', 'F#5', 'F#6', 'F2', 'F5', 'F6', 'G#2', 'G#4', 'G#5', 'G4', 'G5', 'G6']
Mapping of note names to integers: {'0': 0, '0.5': 1, '16th Rest': 2, '16th Triplet (1/6 QL) Rest': 3, '2.7': 4, '3.7': 5, '3.9': 6, '32nd Triplet (1/12 QL) Rest': 7, '5.10': 8, '5.8': 9, '7.0': 10, '8.0': 11, '9.0': 12, 'A2': 13, 'A4': 14, 'A5': 15, 'A6': 16, 'B-4': 17, 'B-5': 18, 'B1': 19, 'B2': 20, 'B3': 21, 'B4': 22, 'B5': 23, 'C#2': 24, 'C#4': 25, 'C#5': 26, 'C1': 27, 'C

### 創建神經網絡的結構 
### LSTM

In [7]:
"""
Define the Neural Network Architecture
"""
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization as BatchNorm, Dropout, Dense, Activation

# Reference for LSTM parameters:
# https://huhuhang.com/post/machine-learning/lstm-return-sequences-state

# Initialize the model
model = Sequential()

# Add the first LSTM layer with 512 units
model.add(LSTM(
    512,
    input_shape=(normalized_input.shape[1], normalized_input.shape[2]),
    recurrent_dropout=0.1,  # Dropout applied to recurrent connections
    return_sequences=True   # Ensure the output is a sequence for stacking
))

# Add the second LSTM layer with 512 units
model.add(LSTM(
    512,
    return_sequences=True,  # Continue outputting sequences
    recurrent_dropout=0.1
))

# Add the third LSTM layer with 512 units (no sequences returned)
model.add(LSTM(512))

# Add a Batch Normalization layer to normalize activations
model.add(BatchNorm())

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.1))

# Add a Dense (fully connected) layer with 256 units and ReLU activation
model.add(Dense(256))
model.add(Activation('relu'))

# Add another Batch Normalization layer
model.add(BatchNorm())

# Add another Dropout layer
model.add(Dropout(0.1))

# Add the final Dense layer with `n_vocab` units and softmax activation
# This maps the output to probabilities for each note
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

# Compile the model with categorical cross-entropy loss and RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Display a summary of the model architecture
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100, 512)       │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 75)             │        19,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 75)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,404,747 (20.62 MB)

 Trainable params: 5,403,211 (20.61 MB)

 Non-trainable params: 1,536 (6.00 KB)

### 訓練神經網絡

In [11]:
"""
Train the neural network for generating music sequences.

This process adjusts the weights of the model based on the provided input
and output, enabling it to learn patterns in the musical dataset. Only the best model
based on training loss will be saved.
"""

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define a callback to save the best model based on training loss
callbacks = [
    # Save only the best model based on minimum loss
    ModelCheckpoint(
        filepath='best_model.keras',  # Filepath to save the best model in .keras format
        monitor='loss',               # Monitor training loss for improvement
        save_best_only=True,          # Save only the best model weights
        mode='min',                   # Minimize the monitored value (loss)
        verbose=1
    ),
    # Stop training early if the loss stagnates
    EarlyStopping(
        monitor='loss', 
        patience=10,                  # Wait for 10 epochs of no improvement
        restore_best_weights=True     # Load the best weights when stopping
    )
]

# Train the model
history = model.fit(
    normalized_input,  # Input data
    network_output,    # Expected output
    epochs=50,         # Total number of training epochs
    batch_size=128,    # Size of each training batch
    callbacks=callbacks,  # Attach callbacks
    verbose=1          # Print progress during training
)

print("Training complete! The best model has been saved as 'best_model.keras'.")


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 3.9462
Epoch 1: loss improved from inf to 3.83662, saving model to best_model.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 181ms/step - loss: 3.9378
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 3.6132
Epoch 2: loss improved from 3.83662 to 3.58728, saving model to best_model.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - loss: 3.6112
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - loss: 3.6192
Epoch 3: loss improved from 3.58728 to 3.53614, saving model to best_model.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - loss: 3.6128
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - loss: 3.4064
Epoch 4: loss improved from 3.53614 to 3.41733, saving model to best_model.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - loss: 3.4072
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - loss: 3.3762
Epoch 5: loss improved from 3.41733 to 3.34470, saving model to best_model.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step

根據選定的音符起始點，從神經網絡預測下一個音符並生成樂譜

In [14]:
"""
Generate Notes Using the Neural Network
"""
import numpy as np

# Randomly select a starting sequence from the training data
start = np.random.randint(0, len(network_input) - 1)
pattern = network_input[start]  # A sequence of length `sequence_length` as the prediction starting point

# Create a dictionary to map integers back to notes
int_to_note = {number: note for number, note in enumerate(pitch_and_rest)}

# Store the generated notes
prediction_output = []

print("Generated notes:")

# Generate 100 notes (adjust range to generate more or fewer notes)
for note_index in range(200):
    # Reshape the pattern to the format expected by the model
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)  # Normalize input data

    # Predict the probabilities of the next note
    prediction = model.predict(prediction_input, verbose=0)

    # Select the note with the highest probability
    index = np.argmax(prediction)

    # Map the integer back to the corresponding note
    result = int_to_note[index]
    print(result)

    # Append the generated note to the output
    prediction_output.append(result)

    # Slide the prediction window: append the new note and drop the oldest one
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


Generated notes:
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
3.7
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
5.10
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
5.10
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
5.10
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
5.10
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
5.10
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
0.5
8.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
5.10
3.7
3.7
3.7
3.7
3.7
3.7
9.0
9.0
9.0
9.0
9.0
9.0


將預測的輸出轉換為音符，並從音符中創建一個MIDI文件 

In [15]:
"""
Convert Predicted Outputs into Notes and Create a MIDI File
"""

from music21 import stream, note, chord, instrument

offset = 0  # Time offset between notes
output_notes = []  # List to store the generated musical elements

# Create note, chord, or rest objects based on the model's generated output
for element in prediction_output:
    # If the element represents a chord (e.g., "60.64.67")
    if ('.' in element) or element.isdigit():
        notes_in_chord = element.split('.')
        notes = []
        for current_note in notes_in_chord:
            try:
                new_note = note.Note(int(current_note))  # Convert to a note object
                new_note.storedInstrument = instrument.Piano()  # Assign an instrument
                notes.append(new_note)
            except ValueError:
                pass  # Skip invalid notes
        new_chord = chord.Chord(notes)  # Create a chord from the notes
        new_chord.offset = offset  # Set the time offset
        output_notes.append(new_chord)

    # If the element represents a single note (e.g., "C4")
    elif element in pitchnames:
        new_note = note.Note(element)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # If the element represents a rest
    elif element in restnames:
        # Convert rest representation to duration if necessary
        new_rest = note.Rest()
        new_rest.quarterLength = rest_to_duration[element]  # Map rest name to duration
        new_rest.offset = offset
        new_rest.storedInstrument = instrument.Piano()
        output_notes.append(new_rest)

    # Increment the offset for the next musical element
    offset += 0.5

# Create a music21 Stream object to hold the notes, chords, and rests
midi_stream = stream.Stream(output_notes)

# Write the Stream to a MIDI file
midi_stream.write('midi', fp='LSTM_generated_music_with_rests.mid')


'LSTM_generated_music_with_rests.mid'